In [ ]:
class Species {
    id: number;
    level: number;
    prey: Species[];
    
    constructor(id: number, level: number, prey: Species[] = []) {
        this.id = id;
        this.level = level;
        this.prey = prey;
    }
}

class Ecosystem {
    levels: Species[][];
    speciationRate: number;
    extinctionProbability: number;
    nextId: number;
    history: { step: number, speciesCount: number[] }[];
    
    constructor(numLevels: number, nichesPerLevel: number, speciationRate: number, extinctionProbability: number) {
        this.levels = Array.from({ length: numLevels }, () => []);
        this.speciationRate = speciationRate;
        this.extinctionProbability = extinctionProbability;
        this.nextId = 1;
        this.history = [];
        
        // 초기 종 생성
        for (let i = 0; i < nichesPerLevel; i++) {
            this.levels[0].push(new Species(this.nextId++, 0));
        }
    }
    
    // 새로운 종 생성 (Origination)
    originate() {
        for (let level = 0; level < this.levels.length; level++) {
            if (Math.random() < this.speciationRate) {
                let preyLevel = level - 1;
                let prey = preyLevel >= 0 ? this.getRandomPrey(preyLevel) : [];
                this.levels[level].push(new Species(this.nextId++, level, prey));
            }
        }
    }
    
    // 멸종 과정 (Extinction)
    extinction() {
        this.levels[0] = this.levels[0].filter(() => Math.random() > this.extinctionProbability);
        
        for (let level = 1; level < this.levels.length; level++) {
            this.levels[level] = this.levels[level].filter(species => species.prey.some(prey => this.levels[prey.level].includes(prey)));
        }
    }
    
    // 주어진 레벨에서 랜덤한 먹이 선택
    getRandomPrey(level: number): Species[] {
        return this.levels[level].length ? [this.levels[level][Math.floor(Math.random() * this.levels[level].length)]] : [];
    }
    
    // 시뮬레이션 실행
    runSimulation(steps: number) {
        for (let i = 0; i < steps; i++) {
            this.originate();
            this.extinction();
            this.history.push({
                step: i + 1,
                speciesCount: this.levels.map(level => level.length)
            });
            // console.log(`Step ${i + 1}: Total species = ${this.getTotalSpecies()}`);
        }

        return this.history.flatMap(d =>
            d.speciesCount.map((speciesCount, i) => ({ step: d.step, level: i, speciesCount }))
        );
    }
    
    getTotalSpecies(): number {
        return this.levels.reduce((sum, level) => sum + level.length, 0);
    }
}

In [ ]:
const ecosystem = new Ecosystem(6, 50, 0.08, 0.01);
const result = ecosystem.runSimulation(1000);
//result

In [ ]:
import { document } from "jsr:@ry/jupyter-helper@0.2.0"
import * as Plot from "https://esm.sh/@observablehq/plot@0.6.17"

Plot.plot({
    marks: [
        Plot.areaY(result, {
            x: "step",
            y: "speciesCount",
            fill: ({ level }) => `Level ${level}`,
        })
    ],
    width: 800,
    height: 400,
    x: { label: "Time Step" },
    y: { label: "Total Species Count" },
    color: { legend: true, scheme: "cividis" },
    document,
})

Level 0 Level 1 Level 2 Level 3 Level 4 Level 5 0 10 20 30 40 50 60 70 ↑ Total Species Count 100 200 300 400 500 600 700 800 900 1,000 Time Step → <path d="M40,130.274L40.741,130.274L41.481,130.274L42.222,139.863L42.963,139.863L43.704,139.863L44.444,139.863L45.185,144.658L45.926,154.247L46.667,159.041L47.407,159.041L48.148,159.041L48.889,163.836L49.63,163.836L50.37,163.836L51.111,163.836L51.852,163.836L52.593,163.836L53.333,168.63L54.074,168.63L54.815,168.63L55.556,168.63L56.296,178.219L57.037,178.219L57.778,178.219L58.519,178.219L59.259,178.219L60,178.219L60.741,178.219L61.481,178.219L62.222,183.014L62.963,192.603L63.704,187.808L64.444,187.808L65.185,197.397L65.926,202.192L66.667,202.192L67.407,206.986L68.148,206.986L68.889,211.781L69.63,206.986L70.37,206.986L71.111,206.986L71.852,206.986L72.593,206.986L73.333,211.781L74.074,211.781L74.815,211.781L75.556,211.781L76.296,211.781L77.037,216.575L77.778,221.37L78.519,221.37L79.259,226.164L80,226.164L80.741,226.164L81.481,226.164L82.222,226.164L82.963,230.959L83.704,235.753L84.444,235.753L85.185,235.753L85.926,235.753L86.667,235.753L87.407,235.753L88.148,240.548L88.889,235.753L89.63,230.959L90.37,230.959L91.111,235.753L91.852,235.753L92.593,235.753L93.333,235.753L94.074,235.753L94.815,235.753L95.556,240.548L96.296,240.548L97.037,240.548L97.778,240.548L98.519,240.548L99.259,240.548L100,240.548L100.741,240.548L101.481,240.548L102.222,240.548L102.963,240.548L103.704,240.548L104.444,240.548L105.185,245.342L105.926,245.342L106.667,245.342L107.407,240.548L108.148,240.548L108.889,240.548L109.63,240.548L110.37,240.548L111.111,240.548L111.852,240.548L112.593,240.548L113.333,240.548L114.074,240.548L114.815,240.548L115.556,240.548L116.296,240.548L117.037,240.548L117.778,240.548L118.519,245.342L119.259,245.342L120,245.342L120.741,245.342L121.481,245.342L122.222,250.137L122.963,254.932L123.704,254.932L124.444,259.726L125.185,269.315L125.926,274.11L126.667,274.11L127.407,278.904L128.148,278.904L128.889,278.904L129.63,278.904L130.37,278.904L131.111,278.904L131.852,278.904L132.593,278.904L133.333,278.904L134.074,278.904L134.815,274.11L135.556,274.11L136.296,278.904L137.037,278.904L137.778,278.904L138.519,278.904L139.259,288.493L140,288.493L140.741,293.288L141.481,293.288L142.222,293.288L142.963,288.493L143.704,288.493L144.444,283.699L145.185,283.699L145.926,288.493L146.667,288.493L147.407,288.493L148.148,288.493L148.889,288.493L149.63,288.493L150.37,288.493L151.111,293.288L151.852,293.288L152.593,293.288L153.333,293.288L154.074,293.288L154.815,293.288L155.556,293.288L156.296,293.288L157.037,293.288L157.778,293.288L158.519,293.288L159.259,293.288L160,293.288L160.741,293.288L161.481,293.288L162.222,293.288L162.963,298.082L163.704,293.288L164.444,293.288L165.185,293.288L165.926,302.877L166.667,302.877L167.407,302.877L168.148,302.877L168.889,302.877L169.63,302.877L170.37,302.877L171.111,302.877L171.852,302.877L172.593,302.877L173.333,302.877L174.074,302.877L174.815,302.877L175.556,302.877L176.296,302.877L177.037,302.877L177.778,302.877L178.519,302.877L179.259,307.671L180,307.671L180.741,307.671L181.481,307.671L182.222,307.671L182.963,307.671L183.704,307.671L184.444,307.671L185.185,307.671L185.926,307.671L186.667,307.671L187.407,307.671L188.148,307.671L188.889,307.671L189.63,307.671L190.37,307.671L191.111,307.671L191.852,307.671L192.593,307.671L193.333,307.671L194.074,307.671L194.815,312.466L195.556,312.466L196.296,317.26L197.037,317.26L197.778,317.26L198.519,317.26L199.259,322.055L200,322.055L200.741,322.055L201.481,322.055L202.222,322.055L202.963,322.055L203.704,322.055L204.444,322.055L205.185,322.055L205.926,326.849L206.667,326.849L207.407,331.644L208.148,331.644L208.889,326.849L209.63,326.849L210.37,326.849L211.111,331.644L211.852,331.644L212.593,331.644L213.333,331.644L214.074,331.644L214.815,331.644L215.556,331.644L216.296,331.644L217.037,331.644L217.778,331.644L218.519,331.644L219.259,336.438L220,336.438L220.741,336.438L221.481,336.438L222.222,336.438L222.963,336.438L223.704,336.438L224.444,336.